# Part 5: 상태 커스터마이징 🎨

## 🎯 목적
메시지 외 업무 데이터까지 다루는 **커스텀 상태**와 **도구 기반 상태 업데이트**를 도입합니다.

### 핵심 개념
- **Custom State Fields**: 메시지 외 검토용/결과용 필드 추가
- **State Updates from Tools**: 도구 결과로 상태 자동 갱신
- **Manual State Updates**: 필요 시 수동으로 상태 수정

### 아키텍처 패턴
```
사용자 입력 → 정보 수집(도구) → 임시 저장(State)
             ↓
           검토/수정 → 최종 결과(State)
```

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import ToolMessage
from langchain_core.tools import InjectedToolCallId, tool
from langgraph.types import Command, interrupt
from langgraph.graph.message import add_messages


# 확장된 State 정의
class CustomState(TypedDict):
    """커스텀 필드가 추가된 상태"""
    messages: Annotated[list, add_messages]
    human_feedback: str  # 사람의 피드백


print("✅ CustomState 정의 완료!")

### 상태 업데이트 도구

도구 실행 결과를 `Command(update=...)` 로 상태에 반영하는 패턴을 사용합니다.

In [ ]:
@tool
def human_review(
    human_feedback: str, 
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Request human review for information."""
    # 인간에게 검토 요청
    human_response = interrupt(
        {"question": "이 정보가 맞나요?", "human_feedback": human_feedback}
    )
    
    feedback = human_response.get("human_feedback", "")
    
    if feedback.strip() == "":
        # 사용자가 AI의 답변에 동의하는 경우
        return Command(
            update={
                "messages": [ToolMessage("확인됨", tool_call_id=tool_call_id)]
            }
        )
    else:
        # 사용자가 수정사항을 제공한 경우
        corrected_information = f"# 사용자에 의해 수정된 피드백: {feedback}"
        return Command(
            update={
                "messages": [
                    ToolMessage(corrected_information, tool_call_id=tool_call_id)
                ],
                "human_feedback": feedback  # 커스텀 필드 업데이트
            }
        )


print("✅ 상태 업데이트 도구 정의 완료!")

### 커스텀 상태 그래프

커스텀 필드를 포함한 `CustomState` 로 그래프를 구성하고, 도구를 통한 상태 업데이트 루프를 적용합니다.

In [ ]:
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI

# 도구 리스트
tools_custom = [human_review]

# 새로운 그래프 구성
custom_graph_builder = StateGraph(CustomState)  # CustomState 사용

# LLM에 도구 바인딩
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_custom_tools = llm.bind_tools(tools_custom)


def chatbot_custom(state: CustomState):
    """커스텀 상태를 사용하는 챗봇"""
    message = llm_with_custom_tools.invoke(state["messages"])
    
    if hasattr(message, "tool_calls"):
        assert len(message.tool_calls) <= 1
    
    return {"messages": [message]}


# 노드와 엣지 추가
custom_graph_builder.add_node("chatbot", chatbot_custom)
tool_node_custom = ToolNode(tools=tools_custom)
custom_graph_builder.add_node("tools", tool_node_custom)

custom_graph_builder.add_conditional_edges("chatbot", tools_condition)
custom_graph_builder.add_edge("tools", "chatbot")
custom_graph_builder.add_edge(START, "chatbot")

# 컴파일
memory_custom = InMemorySaver()
custom_graph = custom_graph_builder.compile(checkpointer=memory_custom)

print("✅ 커스텀 상태 그래프 생성 완료!")

### 커스텀 상태 테스트

`human_review` 도구 호출에서 interrupt 로 중단된 뒤, 재개 시 상태가 올바르게 갱신되는지 확인합니다.

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
import uuid

# 검토 요청하는 질문
user_input = (
    "2024년 노벨 문학상 수상자가 누구인지 조사해주세요. "
    "답을 찾으면 `human_review` 도구를 사용해서 검토를 요청하세요."
)

custom_config = RunnableConfig(configurable={"thread_id": str(uuid.uuid4())})

print(f"User: {user_input}\n")

# 실행 (interrupt에서 중단될 것임)
try:
    result = custom_graph.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=custom_config,
    )
except Exception as e:
    print(f"⏸️ 검토 요청으로 실행이 중단되었습니다")

In [ ]:
# AI가 작성한 내용 확인
last_message = custom_graph.get_state(custom_config).values["messages"][-1]
print("AI가 검토 요청한 내용:")
print("-" * 50)
print(last_message.tool_calls[0]["args"]["human_feedback"])
print("-" * 50)

In [ ]:
# 인간의 검토 응답으로 재개
human_command = Command(
    resume={"human_feedback": "2024년 노벨 문학상 수상자는 대한민국의 한강 작가입니다."}
)

print("\n💡 사람의 수정사항이 반영됩니다...\n")

# 재개
result = custom_graph.invoke(human_command, config=custom_config)
print(f"\n[최종 응답]\n{result['messages'][-1].content}")

---

# Part 6: 상태 이력 관리 ⏰

## 🎯 목적
체크포인트 기반으로 상태를 저장/복원해 **롤백/재실행** 시나리오를 실습합니다.

### 핵심 개념
- **State History**: 상태 변경 이력 관리(추적/복원)
- **Checkpoint ID**: 특정 시점 식별자
- **Rollback**: 지정 시점으로 복원
- **Resume**: 복원 상태에서 재실행

### 기대 효과
- **안전한 실험**, **디버깅 효율**, **A/B 테스트**, **안정성 향상**

### 체크포인트 기반 그래프 구성

상태 이력 확인과 롤백/재실행을 위한 최소 구성으로 시작합니다.

In [ ]:
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.tools import tool
import random

# 간단한 검색 도구 모의
@tool
def search(query: str) -> str:
    """Search for information."""
    # 실제로는 API 호출하지만, 여기서는 모의 결과 반환
    results = [
        "LangGraph는 상태 기반 AI 애플리케이션 프레임워크입니다.",
        "LangGraph는 LangChain 팀이 개발했습니다.",
        "LangGraph는 그래프 구조로 워크플로우를 관리합니다."
    ]
    return random.choice(results)

# 상태 관리 테스트를 위한 체크포인트 기반 그래프
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 도구와 LLM 설정
tools = [search]
llm_with_tools_tt = llm.bind_tools(tools)


def chatbot_tt(state: State):
    """상태 관리 테스트용 챗봇"""
    return {"messages": [llm_with_tools_tt.invoke(state["messages"])]}


# 그래프 구성
graph_builder.add_node("chatbot", chatbot_tt)
tool_node_tt = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node_tt)

graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# 메모리와 함께 컴파일
memory_tt = InMemorySaver()
time_travel_graph = graph_builder.compile(checkpointer=memory_tt)

print("✅ 체크포인트 기반 그래프 생성 완료!")

### 체크포인트 시퀀스 생성

여러 번 대화를 실행해 충분한 상태 이력을 만듭니다.

In [ ]:
time_travel_config = RunnableConfig(configurable={"thread_id": "time-travel-1"})

# 첫 번째 대화
print("[대화 1]")
result1 = time_travel_graph.invoke(
    {"messages": [HumanMessage(content="LangGraph에 대해서 검색해주세요.")]},
    config=time_travel_config,
)
print(f"응답: {result1['messages'][-1].content[:100]}...\n")

In [ ]:
# 두 번째 대화
print("[대화 2]")
result2 = time_travel_graph.invoke(
    {"messages": [HumanMessage(content="LangGraph의 주요 기능을 검색해주세요.")]},
    config=time_travel_config,
)
print(f"응답: {result2['messages'][-1].content[:100]}...")

### 상태 이력 탐색 및 시점 선택

`get_state_history` 로 이력을 조회하고, 롤백/재실행할 체크포인트를 선택합니다.

In [ ]:
# 전체 상태 히스토리 확인
print("📜 상태 히스토리 (최신순):")
print("=" * 80)

# to_replay 변수 초기화
to_replay = None

for i, state in enumerate(time_travel_graph.get_state_history(time_travel_config)):
    print(f"\n[체크포인트 {i}]")
    print(f"  다음 노드: {state.next}")
    print(f"  체크포인트 ID: {state.config['configurable']['checkpoint_id']}")
    print(f"  메시지 수: {len(state.values['messages'])}")
    
    # 첫 번째 검색 완료 시점을 선택
    if len(state.values['messages']) == 4 and to_replay is None:
        print("  ⭐ 이 상태로 되돌아갈 예정")
        to_replay = state
        
        # 마지막 메시지 내용 확인
        last_msg = state.values['messages'][-1]
        if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
            print(f"  도구 호출: {last_msg.tool_calls[0]['name']}")
            print(f"  쿼리: {last_msg.tool_calls[0]['args']}")

print("\n" + "=" * 80)

### 상태 수정 및 최적화

복원된 상태에서 도구 호출 파라미터를 수정해 다른 결과를 유도합니다.

In [ ]:
# 도구 호출 수정 함수
def update_tool_call(message, tool_name, new_args):
    """메시지의 도구 호출을 수정합니다."""
    if hasattr(message, 'tool_calls') and message.tool_calls:
        # 새로운 메시지 생성 (원본 수정 방지)
        from langchain_core.messages import AIMessage
        updated_message = AIMessage(
            content=message.content,
            tool_calls=[{
                "name": tool_name,
                "args": new_args,
                "id": message.tool_calls[0]["id"]
            }]
        )
        return updated_message
    return message

In [ ]:
if to_replay:
    # 원본 도구 호출 확인
    original_message = to_replay.values["messages"][-1]
    print("[원본 도구 호출]")
    print(f"쿼리: {original_message.tool_calls[0]['args']}")
    
    # 도구 호출 수정
    updated_message = update_tool_call(
        original_message,
        tool_name="search",
        new_args={"query": "LangGraph 고급 기능과 best practices"}
    )
    
    print("\n[수정된 도구 호출]")
    print(f"쿼리: {updated_message.tool_calls[0]['args']}")

In [ ]:
# 변경된 메시지를 update_state 로 업데이트
if to_replay:
    updated_state = time_travel_graph.update_state(
        values={"messages": [updated_message]}, 
        config=to_replay.config
    )
    print("✅ 상태가 업데이트되었습니다")

### 수정 상태 재실행

업데이트된 상태로 재실행하여 결과를 비교합니다.

In [ ]:
# 업데이트된 상태에서 재실행
if updated_state:
    print("[재실행 중...]\n")
    result = time_travel_graph.invoke(None, config=updated_state)
    
    print("[수정된 검색으로 얻은 새로운 응답]")
    print("-" * 50)
    print(result['messages'][-1].content)
    print("-" * 50)

## 📚 Part 5-6 요약

### Part 5: 상태 커스터마이징
1. **Custom State Fields**: 메시지 외 비즈니스 데이터 관리
2. **Command Updates**: 도구에서 상태 직접 업데이트
3. **human_review 패턴**: 검토 및 수정 워크플로우
4. **동적 상태 변경**: 런타임에 상태 필드 수정

### Part 6: 상태 이력 관리  
1. **get_state_history**: 모든 체크포인트 조회
2. **Checkpoint 선택**: 특정 시점으로 복원
3. **update_state**: 상태 수정 후 재실행
4. **A/B 테스트**: 다른 파라미터로 비교 실험

### 실무 활용 시나리오
- **디버깅**: 오류 발생 시점으로 롤백하여 원인 분석
- **최적화**: 다양한 파라미터로 재실행하여 최적값 탐색
- **감사**: 모든 상태 변경 이력 추적 및 검증
- **실험**: 안전하게 다양한 시나리오 테스트

### 주의사항
- **메모리 관리**: 체크포인트가 많아지면 저장소 용량 고려
- **동시성**: 여러 사용자가 같은 thread_id 사용 시 충돌 방지
- **보안**: 민감한 상태 정보는 암호화 저장
- **성능**: 프로덕션에서는 PostgresSaver 등 영구 저장소 사용